In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# Import CRUD module
from Shelter_CRUD import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# Database Authentication
username = "aacuser"
password = "AACUSER"
shelter = AnimalShelter(username, password)

# Read all documents in database
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Project 2')

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Dashboard display
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))), # 1st Header
    html.Center(html.B(html.P('Elizabeth Hodgman'))), # 2nd Header
    # Display Grazioso Salvare's Logo - Links to SNHU home page
    html.Center(
        html.A([
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'height':'30%','width':'30%'},
            )
        ],href='https://www.snhu.edu')),
    html.Hr(),
    html.Div(
        # Interactive filter options
        dcc.RadioItems(
            id='radio-items',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'All', 'value': 'reset'}
            ],
            value = 'reset' # Initial filter selected - Starts on "All"
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True, "deselectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Features of interactive data table
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable='single', # Only one row is selectable at a time
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0], # First row initially selected
        page_action="native",
        page_current= 0,
        page_size= 10, # Lists 10 documents per page
        
    ),
    html.Br(),
    html.Hr(),
    # Pie Chart and Map are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
            children=[
                dcc.Graph(
                id='graph-id',
                className='col s12 m6',
                ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Filter Options
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('radio-items', 'value')])                              
def update_dashboard(filter_type):
    # Reset Filter (All)
    if filter_type == 'reset':
        df = pd.DataFrame.from_records(shelter.read({})) # Show all documents
                               
    # Water Rescue Dogs Filter
    elif filter_type == 'water':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
                            
    # Mountain or Wilderness Rescue Dogs Filter
    elif filter_type == 'mountain':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))

    # Disaster or Individual Tracking Dogs Filter
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
        }))
    
    # Update columns to match filter
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data,columns)

# Change column background color if clicked
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Pie Chart
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "data")])
def update_graphs(viewData):
    # Build Pie Chart
    dff = pd.DataFrame.from_records(viewData)
    figure = px.pie(data_frame = dff, names = 'breed', title = "Breed Percentages") # Shows percentages by breed

    return figure

# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', 'derived_virtual_selected_rows')])
def update_map(viewData, derived_virtual_selected_rows):                          
# Code for geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    
    # Get coordinates from selected row
    index = derived_virtual_selected_rows[0]
    posLat = dff.iloc[index, 13] # location_lat
    posLong = dff.iloc[index, 14] # location_long

    # Get name from selected row, replace empty names with "No Name"
    name = dff.iloc[index,9]
    if name == "":
        name = "No Name"
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[posLat, posLong], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup - Points to the location of the animal
            dl.Marker(position=[posLat, posLong], children=[
                dl.Tooltip(dff.iloc[index,4]), # Animal breed - Mouse hover
                dl.Popup([
                    html.H1("Animal Name"), # Animal name - Mouse click
                    html.P(name)
                ])
            ])
        ])
    ]                            

app